In [230]:
import re
import subprocess
import os
import json

import pandas as pd

# import xlwings as xw


In [231]:
exec_name = "pussy"
dir = "out/" + exec_name + "/"
dir_md = dir + "comands-md-files/"


In [232]:
if not os.path.isdir(dir):
    os.mkdir(dir)
    os.mkdir(dir_md)

for f in os.listdir(dir_md):
    os.remove(os.path.join(dir_md, f))


In [233]:
chapters_to_include = [
    "Available Commands:",
    "Usage:",
    "Aliases:",
    "Flags:",
    "Global Flags:",
    "Example:",
    "Examples:",
]


In [244]:
def get_cli_output(command: str, exec_path: str = "/Users/user/go/bin"):

    _args = command.split()
    _args.append("-h")

    out = subprocess.run(_args, capture_output=True, env=dict(PATH=exec_path))
    text = out.stdout.decode("utf-8") + out.stderr.decode("utf-8")

    with open(dir_md + command + ".md", "w") as file:
        file.write(text)

    return text


# text = get_cli_output('cyber')


In [245]:
def parse_cli_stdout(command: str):

    _text = get_cli_output(command)
    _lines = _text.split("\n")

    chapters = {"command": command}
    chapter = "Description"
    chapters[chapter] = []

    for l in _lines:
        if l != "":
            if l in chapters_to_include:
                chapter = l[:-1]  # cutting ":"
                chapters[chapter] = []
            else:
                chapters[chapter].append(l)

    return chapters


# chapters = parse_cli_stdout("gaiad")
# print(chapters)


In [246]:
def get_sub_commands(chapters):
    new_commands = []

    if "Available Commands" in chapters:
        for l in chapters["Available Commands"]:
            if _match := re.search(r"^\s+(.*?)\s+(.*)", l):
                sub_command = _match.group(1)

                command_candidate = " ".join(
                    (chapters["command"] + " " + sub_command).split()
                )
                new_commands.append(command_candidate)

    return new_commands


# new_commands = get_sub_commands(chapters)
# new_commands


In [247]:
parsed_commands = []
all_commands = []
new_commands = [exec_name + " "]

while True:
    for command in new_commands:

        commands_dict = parse_cli_stdout(command)
        new_command_candidates = get_sub_commands(commands_dict)
        all_commands.append(commands_dict)

        for candidate in new_command_candidates:
            if candidate not in parsed_commands:
                new_commands.append(candidate)
                parsed_commands.append(candidate)
        new_commands.remove(command)
    if new_commands == []:
        break


In [ ]:
# all_commands


In [248]:
json_object = json.dumps(all_commands, indent=4)

with open(dir + exec_name + "_commands.json", "w") as outfile:
    outfile.write(json_object)


In [249]:
len(all_commands)


284

In [250]:
command_files = []

for f in os.listdir(dir_md):
    if f[-3:] == ".md":
        command_files.append(f)

command_files.sort()
# command_files


In [251]:
if os.path.exists(dir + exec_name + "_commands.md"):
    os.remove(dir + exec_name + "_commands.md")

for f in command_files:
    with open(dir_md + f) as f2:
        text = f2.read()
        file_content = f"### {f[:-3]}\n\n```\n{text}\n```\n\n"

        with open(dir + exec_name + "_commands.md", "a") as file:
            file.write(file_content)


In [252]:
# with open("all_commands.json", "r") as file:
#     all_commands = json.load(file)


In [253]:
commands = []

for c in all_commands:
    if "Available Commands" not in c:
        commands.append(c)


In [254]:
for c in commands:
    all_flags = []
    all_flags.extend(c["Flags"])
    del c["Flags"]
    if "Global Flags" in c:
        all_flags.extend(c["Global Flags"])
        del c["Global Flags"]

    if all_flags != []:
        c["flags_parsed"] = []
        for b in all_flags:
            if match1 := re.match(
                r"\s\s+(?:-(?P<short>\w)[,\s]+)?(?:--(?P<long>[.\w-]+))\s(?:(?P<format>(uint32|ints|strings|int 64Slice|int|string|uint|float|bytesHex)))?\s*(?P<description>.*)?",
                b,
            ):
                c["flags_parsed"].append(
                    {
                        "long": match1.group("long"),
                        "short": match1.group("short"),
                        "format": match1.group("format"),
                        "description": match1.group("description"),
                    }
                )


In [255]:
for c in commands:
    if len(c["Usage"]) > 1:
        print(c["command"], "command has more than 1 usages", c["Usage"])
    else:
        c["Usage"] = c["Usage"][0]


In [256]:
for entry in commands[:5]:
    print(entry["Usage"])


  pussy collect-gentxs [flags]
  pussy gentx [key_name] [amount] [flags]
  pussy init [moniker] [flags]
  pussy migrate [target-version] [genesis-file] [flags]
  pussy rollback [flags]


In [257]:
usage_test = "'--type=[hash|acc_seq|signature] [hash|acc_seq|signature]'"
re.sub(r"--.*?( |=)(<|\[).*?(>|\])", "", usage_test)[0]


"'"

In [258]:
# geting rid of --flags before neccessary arg
# cyber query tx ['--type=[hash|acc_seq|signature] [hash|acc_seq|signature]']

for c in commands:

    usage = c["Usage"]
    usage = usage.replace("[flags]", "").replace(c["command"], "")
    usage = re.sub(r"--.*?( |=)(<|\[).*?(>|\])", "", usage)
    usage = re.sub(r"\s+", " ", usage).strip()

    c["Usage"] = usage


In [259]:
commands.sort(key=lambda x: x["command"])


In [260]:
# removing optional parameters in brackets ['[name] (--upgrade-height [height]) (--upgrade-info [info])']

arguments = {}

for c in commands:
    clean_string = c["Usage"].split("(")[0]
    arguments[c["command"]] = clean_string.split(" ")

    args = re.findall(r"(?:\[|\<)([\w|\-]+?)(?:\]|\>)(\?)?", clean_string)
    c["args"] = []
    for p in args:
        p = "".join(p)
        p = re.sub(r"[^A-Za-z0-9]", "_", p)
        c["args"].append(p)


In [261]:
def make_fun_name(text: str):
    rsub = re.sub(r"([^a-z])", "-", text)
    return f'"nu-completions-{exec_name}-{rsub}"'


completions_list = []

for c in commands:
    for f in c["flags_parsed"]:
        if m := re.search(r"(\(.*\|.*?\))", f["description"]):
            completions_list.append(m[0])
            f["completions_fun"] = make_fun_name(m[0])


commands[7:10]


[{'command': 'pussy gentx',
  'Description': ['Generate a genesis transaction that creates a validator with a self-delegation,',
   'that is signed by the key in the Keyring referenced by a given name. A node ID and Bech32 consensus',
   'pubkey may optionally be provided. If they are omitted, they will be retrieved from the priv_validator.json',
   'file. The following default parameters are included:',
   '    ',
   '\tdelegation amount:           100000000stake',
   '\tcommission rate:             0.1',
   '\tcommission max rate:         0.2',
   '\tcommission max change rate:  0.01',
   '\tminimum self delegation:     1'],
  'Example': ['$ spacepussy gentx my-key-name 1000000stake --home=/path/to/home/dir --keyring-backend=os --chain-id=test-chain-1 \\',
   '    --moniker="myvalidator" \\',
   '    --commission-max-change-rate=0.01 \\',
   '    --commission-max-rate=1.0 \\',
   '    --commission-rate=0.07 \\',
   '    --details="..." \\',
   '    --security-contact="..." \\',
   ' 

In [262]:

nu_functions_list = []

for i in set(completions_list):
    fun_list = (
        i.replace(" ", "")
        .replace("(", '{ ["')
        .replace("|", '", "')
        .replace(")", '"] }')
    )
    fun_name = make_fun_name(i)
    fun_complete = f"def {fun_name} [] {fun_list}"
    nu_functions_list.append(fun_complete)

nu_completions_functions_string = "\n".join(nu_functions_list)


In [263]:
re.findall(
    r"(\(.*\|.*?\))",
    'Select keyring\'s backend (os|file|kwallet|pass|test) (default "os")',
)


['(os|file|kwallet|pass|test)']

In [264]:
# df = pd.DataFrame(commands)
# pd.json_normalize(df['flags_parsed'])


In [265]:
# df = pd.DataFrame(commands).explode('flags_parsed').reset_index()
# df['command'] = df['command'].apply(lambda x: str(x))
# df2 = pd.json_normalize(df['flags_parsed'])

# df3 = pd.concat([df[['command', 'Usage']], df2], axis=1)
# xw.view(df3)


In [266]:
# for_excel = pd.DataFrame(data=commands).applymap(lambda x: str(x))
# xw.view(for_excel)


In [267]:
# for i in all_commands:
#     print(i["command"], "\n", i["flags_parsed"])


In [268]:
types_nu_dict = {
    "uint": "int",
    "uint32": "int",
    # "float": "decimal",
    "float": "string",
    "strings": "string",
    "bytesHex": "string",
    "ints": "string",
}

get_keys_table = f"{exec_name} _keys table"
get_key_address = f"{exec_name} _keys values"

# nu functions to add at the beginning of the file
functions_list = f"""
export def "{get_keys_table}" [] {{
	{exec_name} keys list | lines | window 5 -s 5 | 
    each {{|it| ($it| parse -r '(?P<col>\w+):(?P<value>.*)')}} | 
    each {{|it| ($it| transpose -r)}} | reduce {{|it, acc| $it | append $acc}} | 
    select name type address 
}}

export def "nu-complete {get_key_address}" [] {{
    ({get_keys_table}).name | zip ({get_keys_table}).address | flatten
  }}

"""

lines = [functions_list, nu_completions_functions_string, "\n"]

for c in commands:
    description = " ".join(c["Description"])[:400]

    lines.append(f"\n# {description}\nexport extern '{c['command']}' [\n")

    for arg in c["args"]:
        lines.append(f"\t{arg}: string\n")

    for flags in c["flags_parsed"]:
        if "." not in flags["long"]:
            long = flags["long"]
            short = "(-" + flags["short"] + ")" if flags["short"] else ""
            f1 = types_nu_dict.get(flags["format"], flags["format"])
            if "completions_fun" in flags:
                f1 = f1 + "@" + flags["completions_fun"]
            format = ": " + f1 if f1 else ""
            description = flags["description"]
            lines.append(f"\t--{long}{short}{format}\t\t# {description}\n")
    lines.append("]\n")

# lines.append("\n}")

lines = [
    l.replace("--from: string", f'--from: string@"nu-complete {get_key_address}"')
    for l in lines
]


with open(dir + exec_name + "_nu_completions.nu", "w") as file:
    file.writelines(lines)
